In [1]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [2]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/Users/riadanas/Desktop/housing regression MLE/.venv/bin/python
3.0.4
/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [3]:
# ==============================================
# 1. Imports
# ==============================================
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost

/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# ==============================================
# 2. Load processed datasets
# ==============================================
train_df = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered_train.csv")
eval_df  = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered_eval.csv")


# Define target + features
target = "price"
X_train, y_train = train_df.drop(columns=[target]), train_df[target]
X_eval, y_eval   = eval_df.drop(columns=[target]), eval_df[target]

print("Train shape:", X_train.shape)
print("Eval shape:", X_eval.shape)

Train shape: (576815, 39)
Eval shape: (148448, 39)


In [5]:
# ==============================================
# 3. Define Optuna objective function with MLflow
# ==============================================
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_eval)
        rmse = float(np.sqrt(mean_squared_error(y_eval, y_pred)))
        mae = float(mean_absolute_error(y_eval, y_pred))
        r2 = float(r2_score(y_eval, y_pred))

        # Log hyperparameters + metrics
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    return rmse

In [6]:
# ==============================================
# 4. Run Optuna study with MLflow
# ==============================================
# Force MLflow to always use the root project mlruns folder
mlflow.set_tracking_uri("/Users/riadanas/Desktop/housing regression MLE/mlruns")
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)

[I 2025-09-01 15:36:14,579] A new study created in memory with name: no-name-f5b3ffa0-6b7d-4219-bd74-4566338187ec
[I 2025-09-01 15:36:31,309] Trial 0 finished with value: 71950.57496746138 and parameters: {'n_estimators': 944, 'max_depth': 9, 'learning_rate': 0.16003453198107737, 'subsample': 0.8422380573883641, 'colsample_bytree': 0.5161385418391814, 'min_child_weight': 9, 'gamma': 2.9538531328406576, 'reg_alpha': 1.617607263140248e-08, 'reg_lambda': 0.7918676729679146}. Best is trial 0 with value: 71950.57496746138.
[I 2025-09-01 15:36:37,127] Trial 1 finished with value: 73594.79644066688 and parameters: {'n_estimators': 455, 'max_depth': 7, 'learning_rate': 0.07931313314345592, 'subsample': 0.7242773979057299, 'colsample_bytree': 0.9071349492451612, 'min_child_weight': 3, 'gamma': 4.134267433553411, 'reg_alpha': 0.00010166645591781573, 'reg_lambda': 0.0006480122633956456}. Best is trial 0 with value: 71950.57496746138.
[I 2025-09-01 15:36:47,062] Trial 2 finished with value: 69731.

Best params: {'n_estimators': 504, 'max_depth': 10, 'learning_rate': 0.019582485225566305, 'subsample': 0.5820741225928057, 'colsample_bytree': 0.7441951900617425, 'min_child_weight': 8, 'gamma': 3.9323382198505197, 'reg_alpha': 0.009074339702843045, 'reg_lambda': 0.00017288191340220137}


In [7]:
# ==============================================
# 5. Train final model with best params and log to MLflow
# ==============================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_eval)

mae = mean_absolute_error(y_eval, y_pred)
rmse = np.sqrt(mean_squared_error(y_eval, y_pred))
r2 = r2_score(y_eval, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log final model
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    mlflow.xgboost.log_model(best_model, name="model")

Final tuned model performance:
MAE: 31376.607367309574
RMSE: 71378.51620394755
R²: 0.9606273122493897


/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [15:38:38] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/09/01 15:38:42 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/09/01 15:38:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
